# Module 4 Adversarial FL – Outline

## 1. Federated Baseline Imports

In [ ]:
from copy import deepcopy
import sys
from pathlib import Path

import yaml
import numpy as np
import torch

MODULE_DIR = Path.cwd() / "4_Adversarial_FL"
if MODULE_DIR.exists() and str(MODULE_DIR) not in sys.path:
    sys.path.append(str(MODULE_DIR))

from util_functions import set_seed, evaluate_fn, run_fl
from load_data_for_clients import dist_data_per_client
from algos import (
    Server,
    ScaffoldServer,
    FedAdamServer,
    FedAdagradServer,
    FedYogiServer,
)


## 2. Federated Baseline Paths & Config

In [ ]:
CONFIG_PATH = MODULE_DIR / "config.yaml"
if not CONFIG_PATH.exists():
    raise FileNotFoundError(f"Expected config at {CONFIG_PATH}")

with CONFIG_PATH.open() as f:
    CONFIG = yaml.safe_load(f)

global_config = CONFIG.get("global_config", {})
data_config = CONFIG.get("data_config", {})
model_config = CONFIG.get("model_config", {})
alg_configs = CONFIG.get("algorithms", {})
attack_defaults = CONFIG.get("attack", {})

set_seed(global_config.get("seed", 42))
AVAILABLE_ALGORITHMS = list(alg_configs)
print("Loaded config:", CONFIG_PATH)
print("Available algorithms:", AVAILABLE_ALGORITHMS)


## 3. Federated Baseline Helpers

In [ ]:
ALGORITHM_MAP = {
    "FedAvg": Server,
    "Scaffold": ScaffoldServer,
    "FedAdam": FedAdamServer,
    "FedAdagrad": FedAdagradServer,
    "FedYogi": FedYogiServer,
}

missing = sorted(set(AVAILABLE_ALGORITHMS) - set(ALGORITHM_MAP))
if missing:
    raise KeyError(f"No server mapping registered for: {missing}")


def train_server(alg_name: str, attack_cfg: dict | None = None):
    if alg_name not in alg_configs:
        raise ValueError(f"Algorithm {alg_name!r} not found in configuration.")

    alg_conf = alg_configs[alg_name]
    fed_cfg = deepcopy(alg_conf["fed_config"])
    fed_cfg["algorithm"] = alg_name
    optim_cfg = deepcopy(alg_conf.get("optim_config", {}))
    attack_cfg = deepcopy(attack_cfg or {"malicious_fraction": 0.0})

    return run_fl(
        ALGORITHM_MAP[alg_name],
        global_config,
        data_config,
        fed_cfg,
        model_config,
        optim_cfg,
        attack_cfg,
    )


def summarise_server(server) -> dict:
    loss, acc = evaluate_fn(server.data, server.x, server.criterion, server.device)
    history = server.results if hasattr(server, "results") else {}
    return {
        "final_loss": float(loss),
        "final_accuracy": float(acc),
        "history": {
            "loss": list(history.get("loss", [])),
            "accuracy": list(history.get("accuracy", [])),
        },
    }


def run_one_algorithm(alg_name: str, attack_cfg: dict | None = None) -> dict:
    server = train_server(alg_name, attack_cfg=attack_cfg)
    summary = summarise_server(server)
    del server
    torch.cuda.empty_cache()
    return summary


def run_all_algorithms(
    algorithms: list[str] | None = None,
    attack_cfg: dict | None = None,
) -> dict:
    algorithms = algorithms or AVAILABLE_ALGORITHMS
    results: dict[str, dict] = {}
    for name in algorithms:
        results[name] = run_one_algorithm(name, attack_cfg=attack_cfg)
    return results


## 4. Federated Baseline Runs

In [ ]:
BASELINE_ALGORITHMS = ["FedAvg"]  # adjust this list to sweep additional algorithms

baseline_results = run_all_algorithms(BASELINE_ALGORITHMS)
baseline_results


## 5. Surrogate Imports

In [ ]:
# TODO: import surrogate-specific modules or utilities


## 6. Surrogate Paths & Config

In [ ]:
# TODO: pull surrogate training configuration


## 7. Surrogate Baseline

In [ ]:
# TODO: train or load the surrogate-only baseline


## 8. Baseline Comparison

In [ ]:
# TODO: compare surrogate metrics against federated baselines


## 9. Attack Imports

In [ ]:
# TODO: import attack utilities or supporting modules


## 10. Attack Paths & Config

In [ ]:
# TODO: load attack-related configuration blocks


## 11. Attack Implementations

In [ ]:
# TODO: define PGD/FGSM/random-noise routines in-notebook


## 12. Malicious Client Definition

In [ ]:
# TODO: implement or override malicious client behaviour


## 13. Attack Execution Helpers

In [ ]:
# TODO: add helpers to run one attack or sweep attacks


## 14. Surrogate Attack Experiments

In [ ]:
# TODO: craft adversarial batches and report surrogate metrics


## 15. Federated Attack Sweeps

In [ ]:
# TODO: run attacks against the FL server under different settings
